In [ ]:
from torchvision import datasets
from models import Vgg16, AlexNet
import random 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import tensorflow as tf

from copy import deepcopy
import itertools

from methods import data_handler
from PIL import Image

## Load test data

In [ ]:
img_folder = datasets.ImageFolder(root=r'C:\Users\julia\Dokumente\GitHub\development\data\imagenetv2-matched-frequency-format-val')

In [ ]:
def sample_random_images(img_folder, number, seed=None):
    images = []
    img_paths = img_folder.imgs.copy()
    if seed:
        random.seed(seed)
    for i in range(number):
        rand_idx = random.randint(0, len(img_paths))
        rand_img_path = img_paths.pop(rand_idx)[0]
        rand_img = img_folder.loader(rand_img_path)
        # resize
        rand_img = rand_img.resize((224, 224))
        # convert to np array
        images.append(np.array(rand_img))
    return np.array(images) / 255

In [ ]:
def load_entire_val_set(img_folder):
    """
    So liegen alle Bilder als np array vor; müssen aber als torch-tensor-vorliegen vorliegen
    """
    images = []
    labels_idx = []
    for img_path in tqdm(img_folder.imgs):
        rand_img = img_folder.loader(img_path[0])
        # resize
        rand_img = rand_img.resize((224, 224))
        # convert to np array
        images.append(np.array(rand_img))
        labels_idx.append(int(img_path[0].split("\\")[-2]))
    return np.array(images), np.array(labels_idx)

In [ ]:
images, labels_idx = load_entire_val_set(img_folder)

In [ ]:
labels = data_handler.get_labels()

## Predict test data

In [ ]:
vgg = Vgg16()
alexnet = AlexNet()

In [ ]:
model = tf.keras.applications.VGG16(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000
)

In [ ]:
IMG_IDX = 1100

In [ ]:
input_preprocessed = tf.keras.applications.vgg16.preprocess_input(images[IMG_IDX].reshape((-1, ) + images[0].shape))

In [ ]:
out_idx = model.predict(input_preprocessed).argmax()
out_idx

In [ ]:
images[IMG_IDX]

In [ ]:
test = r"C:\Users\julia\Dokumente\GitHub\development\data\imagenetv2-matched-frequency-format-val\900\d9885ca6d7331dec3d8a37283d0b981f7e5892cb.jpeg"

In [ ]:
"LIME" is not "LIME"

In [ ]:
plt.imshow(images[IMG_IDX])

In [ ]:
img_folder.imgs

In [ ]:
preds = []
labels_idx = []
for img_path in tqdm(img_folder.imgs):
    img = img_folder.loader(img_path[0])
    
    # for PyTorch
#     input_tensor = data_handler.transform()(img)
#     input_tensor = input_tensor.unsqueeze(0)
#     output = model.predict(input_tensor)
#     The output has unnormalized scores. To get probabilities, you can run a softmax on it.
#     probabilities = torch.nn.functional.softmax(output[0], dim=0)
    
    # for tensorflow/keras
    img = img.resize((224, 224), Image.BICUBIC)
    img = np.array(img)
    input_preprocessed = tf.keras.applications.vgg16.preprocess_input(img.reshape((-1, ) + img.shape))
    probabilities = model.predict(input_preprocessed)
    
    labels_idx.append(int(img_path[0].split("\\")[-2]))
    preds.append(probabilities)

## Create statistics

In [ ]:
"""
Anmerkung: finde noch Klassen die sich ähneln heraus

Was kann man für Stats berechnen:
- Pro Klasse Prozent richtig oder falsch
- für falsche Preds: 
    - Berechne Differenz zwischen falscher prob und eigentlich richtiger Prob
        --> um wie viel % lag pred daneben
        --> Könnte bei großer Differenz auch als negativ BSP für Class Confidence gelten
            (ist sowas überhaupt erwünscht; kommt wahrscheinlich darauf an, wie oft solche
            großen Abweichungen vorkommen)
        --> hier sollten generell die Erklärungen besonders schlecht ausfallen
        --> falsche Preds mit hoher Differenz sind besonders für Klassen interessant,
            bei denen ansonsten viel richtig predicted wird (eventuell Clever-Hans)
- für richtige Preds
    - Berechne Differenz zwischen richtiger Pred und nächstgrößter Pred
        --> mögliches Identifizieren schwieriger/ähnlicher Klassen (als schwieriges
        BSP für Conf Scores)
        --> m
- 
"""

In [ ]:
df["pred_conf"] = df.apply(lambda x: x.max(), axis=1)
df["pred_label"] = df[[0:1000]].apply(lambda x: x.argmax(), axis=1)
df["true_label_idx"] = labels_idx
df["pred_is_correct"] = df["pred"] == df["true_label_idx"]

In [ ]:
def get_prediction_success_stats_df(preds, labels_idx):
    df = pd.DataFrame(np.array(preds))
    df["pred_conf"] = df[df.columns[0:1000]].apply(lambda x: x.max(), axis=1)
    df["pred_label"] = df[df.columns[0:1000]].apply(lambda x: x.argmax(), axis=1)
    df["true_label_idx"] = labels_idx
    df["pred_is_correct"] = df["pred_label"] == df["true_label_idx"]
    return df

In [ ]:
def calc_diff(x):
    # if pred correct, calc confidence differnce to nearest class
    if x["pred_is_correct"]:
        preds = x[0:1000].sort_values(ascending=False)
        return preds[0] - preds[1]
    # else calc confidence difference between predicted class and true class
    conf_true_class = x[x["true_label_idx"]]
    return x[0:1000].max() - conf_true_class

In [ ]:
def get_class_grouped_success_stats_df(df_stats):
    # calc accuracy per class
    df_class_grp = df_stats.groupby(by="true_label_idx")["pred_is_correct"].sum() / 10
    # calc mean accuracies per class and pred_is_correct
    # df_vgg_mean_conf = df_stats.groupby(by=["true_label_idx", "pred_is_correct"])["pred_conf"].mean().to_frame()
    return df_class_grp

In [ ]:
a = [pred[0] for pred in preds ]

In [ ]:
np.array(a).shape

In [ ]:
df = get_prediction_success_stats_df(a, labels_idx)
df.to_pickle("./data/stats/df_vgg_tf.pickle")

In [ ]:
df

In [ ]:
df["pred_is_correct"].sum() / df.shape[0]

In [ ]:
df_vgg["pred_is_correct"].sum() / df_vgg.shape[0]

### Vgg stats

In [ ]:
df_vgg = pd.read_pickle("./data/stats/df_vgg.pickle")

In [ ]:
# nachträglich berechnen
# df_vgg["conf_diff"] = df_vgg.apply(calc_diff, axis=1) # bis jetzt nicht mit anzufangen
df_vgg["pred_conf"] = df_vgg.apply(lambda x: x[0:1000].max(), axis=1)

In [ ]:
# overall accuray:
df_vgg["pred_is_correct"].sum() / 10000

In [ ]:
df_vgg_grp = get_class_grouped_success_stats_df(df_vgg)

In [ ]:
df_vgg_grp

In [ ]:
df_vgg_grp.plot()

In [ ]:
df_vgg_frac_correct = df_vgg_grp.value_counts(normalize=False).sort_index().to_frame("fraction_correct").reset_index()
df_vgg_frac_correct.columns = ["accuracy", "number classes"]

In [ ]:
df_vgg_frac_correct

In [ ]:
# print unsuccessful classes
for idx in df_vgg_grp[df_vgg_grp == 0.0].index:
    print(labels[str(idx)][1])

In [ ]:
# print successful classes
for idx in df_vgg_grp[df_vgg_grp == 1.0].index:
    print(labels[str(idx)][1])

### Alex stats

In [ ]:
df_alex = pd.read_pickle("./data/stats/df_alexnet.pickle")

In [ ]:
# nachträglich berechnen
# df_vgg["conf_diff"] = df_vgg.apply(calc_diff, axis=1) # bis jetzt nicht mit anzufangen
df_alex["pred_conf"] = df_alex.apply(lambda x: x[0:1000].max(), axis=1)

In [ ]:
# overall accuray:
df_alex["pred_is_correct"].sum() / 10000

In [ ]:
df_alex_grp = get_class_grouped_success_stats_df(df_alex)

In [ ]:
df_alex_grp

In [ ]:
df_alex_grp.plot()

In [ ]:
df_alex_frac_correct = df_alex_grp.value_counts(normalize=False).sort_index().to_frame("fraction_correct").reset_index()
df_alex_frac_correct.columns = ["accuracy", "number classes"]

In [ ]:
df_alex_frac_correct

In [ ]:
# print unsuccessful classes
for idx in df_alex_grp[df_alex_grp == 0.0].index:
    print(labels[str(idx)][1])

In [ ]:
# print successful classes
for idx in df_alex_grp[df_alex_grp == 1.0].index:
    print(labels[str(idx)][1])

### Analyze similarities and differences

In [ ]:
df_diff = (df_vgg_grp - df_alex_grp)

In [ ]:
df_diff.plot(xlabel="class_index", ylabel="accuracy_vgg - accuracy-alex", title="VGG16 vs. AlexNet on all classes")

In [ ]:
df_diff.sort_values().head(5)

In [ ]:
df_diff.sort_values().tail(5)

In [ ]:
num = 5
for i in range(num):
    print(labels[str(df_diff.sort_values().head(5).index[i])][1])

In [ ]:
num = 5
for i in range(num):
    print(labels[str(df_diff.sort_values().tail(5).index[i])][1])

# Picking  procedure as proposed by Christian

In [ ]:
imgs_idx = list(range(10000))
xai_methods = ['gradCam', 'LRP', 'SHAP', 'LIME', 'ConfidenceScores', 'CEM']
models = ["alex", "vgg"]

df_vgg = pd.read_pickle("./data/stats/df_vgg.pickle")
df_alex = pd.read_pickle("./data/stats/df_alexnet.pickle")

In [ ]:
def get_fixed_img_questionnaires(imgs_idx, xai_methods, models):
    
    NUM_QUESTIONNAIRES = 12
    NUM_IMGS = 12
    questionnaires_list = []
    random_imgs_idx = [imgs_idx.pop(random.randint(0, len(imgs_idx)-1)) for i in range(NUM_IMGS)]
    permutations = list(itertools.product(random_imgs_idx, models, xai_methods))
    # distribute permutations on questionnaires 
    for q in range(NUM_QUESTIONNAIRES):
        questionnaire = []
        for i in range(NUM_IMGS):
            if (q+i) > (NUM_IMGS-1):
                questionnaire.append(permutations[i*NUM_IMGS:i*NUM_IMGS+NUM_IMGS][(q+i) - NUM_IMGS])
            else:
                questionnaire.append(permutations[i*NUM_IMGS:i*NUM_IMGS+NUM_IMGS][q+i])
        random.shuffle(questionnaire)
        questionnaires_list.append(questionnaire)
    
    return questionnaires_list

In [ ]:
questionnaires_list = get_fixed_img_questionnaires(imgs_idx, xai_methods, models)

In [ ]:
def add_random_unique_images(questionnaires_list, imgs_idx, df_alex, df_vgg, xai_methods):
   
    FINAL_QUESTIONNAIRE_SIZE = 24
    
    for idx_qn, questionnaire in enumerate(questionnaires_list):
        
        df_variants_count = pd.DataFrame(list(itertools.product(xai_methods, models, [True, False]))).groupby([0, 1, 2]).count()
        df_variants_count["count"] = 0
        
        # evaluate variants for the already drawn fixed questions
        for idx_q, question in enumerate(questionnaire):
            if question[1] == "alex":
                if df_alex["pred_is_correct"][question[0]]:
                    questionnaires_list[idx_qn][idx_q] += (True, )
                    df_variants_count.loc[question[2], "alex", True]["count"] += 1
                else: 
                    questionnaires_list[idx_qn][idx_q] += (False, )
                    df_variants_count.loc[question[2], "alex", False]["count"] += 1
            else:
                if df_vgg["pred_is_correct"][question[0]]:
                    questionnaires_list[idx_qn][idx_q] += (True, )
                    df_variants_count.loc[question[2], "vgg", True]["count"] += 1
                else:
                    questionnaires_list[idx_qn][idx_q] += (False, )
                    df_variants_count.loc[question[2], "vgg", False]["count"] += 1
                    
        # add addtional random images to each questionnaire such that for every variant in df_variants_count the 
        # count will be 1
        while df_variants_count["count"].sum() != FINAL_QUESTIONNAIRE_SIZE:
            rand_img_idx = imgs_idx.pop(random.randint(0, len(imgs_idx)-1))
            
            alex_pred = df_alex.loc[rand_img_idx]["pred_is_correct"]
            vgg_pred = df_alex.loc[rand_img_idx]["pred_is_correct"]
            
            df_alex_options = df_variants_count.loc[:, "alex", alex_pred]
            df_alex_options = df_alex_options[df_alex_options["count"] == 0]
            
            df_vgg_options = df_variants_count.loc[:, "vgg", vgg_pred]
            df_vgg_options = df_vgg_options[df_vgg_options["count"] == 0]
            
            if not df_alex_options.empty:
                rand_variant = df_alex_options.index[random.randint(0, df_alex_options.shape[0]-1)]
                question = (rand_img_idx, rand_variant[1], rand_variant[0], rand_variant[2])
                questionnaire.append(question)
                df_variants_count.loc[rand_variant]["count"] += 1
            
            elif not df_vgg_options.empty:
                rand_variant = df_vgg_options.index[random.randint(0, df_vgg_options.shape[0]-1)]
                question = (rand_img_idx, rand_variant[1], rand_variant[0], rand_variant[2])
                questionnaire.append(question)
                df_variants_count.loc[rand_variant]["count"] += 1
                    
    return df_variants_count

In [ ]:
df_test = add_random_unique_images(questionnaires_list, imgs_idx, df_alex, df_vgg, xai_methods)

In [ ]:
df_qn1 = pd.DataFrame(questionnaires_list[0])

In [ ]:
df_qn1

In [ ]:
df_qn1_count  = df_qn1.groupby(by=[2,1,3]).count()

In [ ]:
df_qn1_count.index.name = ["XAI_Method", "model", "outcome", "image_count"]

In [ ]:
df_qn1_count.columns = ["XAI_Method", "model", "outcome", "image_count"]

In [ ]:
df_qn1_count

## Writing question text including label to files

In [ ]:
import os
from methods import data_handler
from torchvision import datasets

In [ ]:
questionnaires_list = data_handler.get_questionnaires("data/questionnaires.pickle")

In [ ]:
questionnaires_list[1]

In [ ]:
img_folder = datasets.ImageFolder(root=r'/Users/julianvonklitzing/Documents/GitHub/development/data/imagenetv2-matched-frequency-format-val')

In [ ]:
labels = data_handler.get_labels()

In [ ]:
img_folder.imgs[0][0].split("/")[-2]

In [ ]:
current_dir = os.getcwd()
folder_path = os.path.join(current_dir, f"questionnaire_forms_questions_txts")
    
if not os.path.exists(folder_path):
        os.mkdir(folder_path)

for idx, questionnaire in enumerate(questionnaires_list):
    file1 = open(os.path.join(folder_path, f"questionnaire_{idx+1}.txt"), "w") # write mode
    for qu_idx, question in enumerate(questionnaire):
        label_idx_str = img_folder.imgs[question[0]][0].split("/")[-2]
        label = labels[label_idx_str][1]
        print(label)
        file1.write(f"{qu_idx + 1}.) Do you think the AI has recognized the '{label.replace('_', ' ')}' in this image?\n")
    file1.close()

In [ ]:
"test_split".replace("_", " ")

In [ ]:
df_alex = pd.read_pickle("./data/stats/df_alexnet.pickle")

In [ ]:
df_alex_2 = pd.read_pickle("./data/stats/df_alexnet_2.pickle")

In [ ]:
df_alex["pred_is_correct"].sum() / df_alex.shape[0]

In [ ]:
df_alex_2["pred_is_correct"].sum() / df_alex.shape[0]

In [ ]:
df_vgg = pd.read_pickle("./data/stats/df_vgg.pickle")
df_vgg_2 = pd.read_pickle("./data/stats/df_vgg_2.pickle")

In [ ]:
df_vgg["pred_is_correct"].sum() / df_alex.shape[0]

In [ ]:
df_vgg_2["pred_is_correct"].sum() / df_alex.shape[0]

In [ ]:
import pandas as pd

In [ ]:
df_vgg_2 = pd.read_pickle("./data/stats/df_vgg.pickle")

In [ ]:
df_vgg_2["pred_is_correct"].sum()/df_vgg_2.shape[0]

## Confidence Scores

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
labels = ["1st most probable class\n (the predicted class)", "2nd most probable class", "3rd most probable class"]
numbers = [0.002, 90.5784, 30]
numbers.sort(reverse=True)

In [ ]:
plt.figure(figsize=(8, 4))
splot=sns.barplot(x=numbers,y=labels, color="blue")
plt.xlim(0, 100)
plt.xlabel("AI's confidence for the given classes \n in %", size=20)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.bar_label(splot.containers[0], labels=[f"{round(score, 1)}%" if round(score, 1) >= 0.1 else "smaller than 0.1%" for score in numbers], size=13)
plt.tight_layout()
fig = plt.gcf()
plt.close()

fig.savefig("test.png", )# bbox_inches='tight'

In [ ]:
fig.show()

In [ ]:
plt.xlim(0, 100)
plt.barh(labels, numbers, height=0.5)
plt.xlabel("Confidence of the model for the given classes in %")
plt.xticks()
plt.yticks(fontsize=20)
#plt.bar_label(["t1", "t2", "t3"])
plt.tight_layout()

# Automate html code for survey

## Automate html for last side

In [ ]:
import os
import pickle

In [ ]:
def get_questionnaires(path):
    with open(path,'rb') as f:
        questionnaires_list = pickle.load(f)
    
    return questionnaires_list

In [ ]:
files = os.listdir("/Users/julianvonklitzing/Desktop/files")

In [ ]:
questionnaires_list = get_questionnaires("data/questionnaires.pickle")

In [ ]:
files[50].split("_")

In [ ]:
start_questionnaire = 6
end_questionnaire = 12


START = """<h1>Thank you for completing our questionnaire!</h1>
<p>We would like to thank you very much for advancing our research.</p>
<p>Your answers were transmitted, you may close the browser window or tab now.</p>
<p>If you want to be informed about the outcomes of our survey, reach out to our <a href="mailto:xai_survey@ITTuberlin.onmicrosoft.com">Shared Mailbox</a>.
<p> Below, you can find the AI predictions to each image:</p><br>"""


END = """<!-- Additional spacing (remove or adjust as required)  -->
            <div style="padding-bottom: 120px"></div>"""



for i in range(start_questionnaire, end_questionnaire+1):
    f = open(f"{i}_end.txt", "w")
    f.write(f"{START} \n")
    corr_count=0
    wrong_count=0
    for j in range(1, 24+1):
        try:
            file = [file for file in files if file.startswith(f"{i}_{j}_")][0]
            
            xai = file.split("_")[4]
            width = "width=600" if xai=="cs" else ""
            
            if file.split("_")[3].startswith("t"):
                corr_count+=1
            else:
                wrong_count+=1
                
            correct = "CORRECT" if file.split("_")[3].startswith("t") else "WRONG"
            varibale = f'<center><img src="{file}" {width}> <b><center>AI Prediction: {correct}</center><hr>'
            f.write(f"{varibale} \n")
            f.write(f"\n")
        except:
            f.write(f" ----------------FAIL-------------- \n")
            f.write(f"\n")
            print(f"FAIL QN:{i}  Q:{j}")
            
    print(f"QN {i}: CORRECT:{corr_count}  WRONG:{wrong_count}")        
    f.write(f"{END} \n")
    f.close()


## Automate html for question-side

## Automate image preparation

In [ ]:
"""
Two matplotlib figures must be put into one single (1, 2) Plot
    - right one with colotbar or ConfidenceScore plot
"""

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps

In [ ]:
# https://note.nkmk.me/en/python-pillow-concat-images/
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst


def get_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

In [ ]:
folder_path = "/Users/julianvonklitzing/Documents/GitHub/development/data/images/0"
# folder_path = "/Users/julianvonklitzing/Documents/GitHub/development/questionnaire_forms_08-06_23-10/questionnaire_1"
examp_imgs_paths = [f"{folder_path}/{file}" for file in os.listdir(folder_path)]

In [ ]:
im1 = Image.open(examp_imgs_paths[1])
im2 = Image.open(examp_imgs_paths[2])

im1 = im1.resize((224, 224))

In [ ]:
im1

In [ ]:
im2 = im2.resize((224, 224))

In [ ]:
im2

In [ ]:
get_concat_h(im2, im1)
# get_concat_v(im1, im1).save('data/dst/pillow_concat_v.jpg')

In [ ]:
plt.imshow(im1)
plt.axis("off")
plt.title("Image of a lion")
fig1 = plt.gcf()
fig1.set_figwidth(244)

In [ ]:
fig1

In [ ]:
plt.imshow(im2)
plt.axis("off")# location="bottom"
fig2 = plt.gcf()

In [ ]:
fig2.colorbar()

### New plotting approach

In [ ]:
"""
Vielleicht einfachster Approach:

erstelle subplots und übergebe alle Information (orginales Bild und Erklrung in Array-Form; plotte erst dort)
SAHP: shap_values
Integrated Gradient: attr_ig
LIME: heatmap
LRP: denke "b" aus heatmap() funktion
gradCAM: grayscale_cam ??
"""

In [ ]:
im1 = Image.open(examp_imgs_paths[1]).resize((224, 224))
im1_gray = gray_image = ImageOps.grayscale(im1)
im2 = Image.open(examp_imgs_paths[2]).resize((224, 224))

In [ ]:
im1_np = np.array(im1.getdata()).reshape(im1.size[0], im1.size[1], 3)
im1_gray_np = np.array(im1_gray.getdata()).reshape(im1_gray.size[0], im1_gray.size[1])

im2_np = np.array(im2.getdata()).reshape(im2.size[0], im2.size[1], 3)

In [ ]:
plt.imshow(im1_np, cmap=gray)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(im1_np)
ax1.axis("off")
ax1.set_title("Image of a lion")
ax2.imshow(im1_gray_np, cmap="gray", alpha=0.4) # im1_gray_np
ax2.imshow(im2_np, alpha=0.3)
ax2.axis("off")

In [ ]:
plt.imshow(np.random.random((224, 224)) * 255)

In [ ]:
questionnaires_list[11]

## Trying stuff

In [ ]:
from methods import data_handler, gradcam, LRP, SHAP, lime, integrated_gradients, confidence_scores
from models import AlexNet, Vgg16
import argparse
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import torch
from datetime import datetime
import random

In [ ]:
# define models
#models_list = [Vgg16(), AlexNet()]
#[model.eval() for model in models_list]

labels = data_handler.get_labels()
model = AlexNet()
model.eval()
for img_idx in range(1):

    img_org_np, img_prep_torch, img_name, img_true_label_str = data_handler.get_question_image(
            r'/Users/julianvonklitzing/Documents/GitHub/development/data/images',
            img_idx,
            labels)

    # for model in models_list:
    #     model_used = model
        
    # predict
    output = model.predict(img_prep_torch)
    # output has unnormalized scores. To get probabilities, run a softmax on it.
    pred_idx = torch.nn.functional.softmax(output[0], dim=0).detach().numpy().argmax()
    label = labels[str(pred_idx)]
    # must be manually verfied if true, because there are no true labels available for manually
    # downloaded images
    print(f"{img_name}, {model.name}: {label[1]}")


    # gradcam.explain(model_used.model, img_prep_torch, img_org_np).savefig(os.path.join("introduction" , f"intro_gradCAM_{model.name}_{img_name}"))
    # LRP.explain(model_used.model, img_prep_torch, img_name, model_used.name).savefig(os.path.join("introduction", f"intro_LRP_{model.name}_{img_name}"))
    # lime_ex = lime.LIMEExplainer(model_used)
    # lime_ex.explain(img_org_np).savefig(os.path.join("introduction", f"intro_LIME_{model.name}_{img_name}"))
    shap_values = SHAP.explain(model_used.model, img_prep_torch, img_org_np, labels).savefig(os.path.join("introduction", f"intro_SHAP_{model.name}_{img_name}"))
    print(fig)
    #ige = integrated_gradients.IntegratedGradientsExplainer(model_used)
    # ige.explain(img_prep_torch).savefig(os.path.join("introduction", f"intro_IntegratedGradients_{model.name}_{img_name}"))
    #confidence_scores.explain(model_used, img_prep_torch, labels, 3).savefig(os.path.join("introduction", f"intro_ConfidenceScores_{model.name}_{img_name}"))


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(im1_np)
ax1.axis("off")
ax1.set_title("Image of a lion")
ax2.imshow(im1_gray_np, cmap="gray", alpha=0.4) # im1_gray_np
ax2.imshow(im2_np, alpha=0.3)
ax2.axis("off") 

## Shuffle question order

In [ ]:
from methods import data_handler
from torchvision import datasets
import os
from tqdm import tqdm

In [ ]:
labels = data_handler.get_labels()

In [ ]:
def img_idx_to_label(img_idx, labels, testset_path):
    img_folder = datasets.ImageFolder(root=testset_path)
    img_path = img_folder.imgs[img_idx][0]
    # extract correct class
    class_idx_true_str = img_path.split(os.sep)[-2]
    img_label_true = labels[class_idx_true_str][1]
    return img_label_true

In [ ]:
questionnaires_org = data_handler.get_questionnaires("data/questionnaires.pickle")
questionnaires_shuffled = data_handler.get_questionnaires("data/questionnaires_shuffled.pickle")

In [ ]:
img_idx_to_label(9521, labels, "data/imagenetv2-matched-frequency-format-val")

In [ ]:
("custard_apple", ) + questionnaires_shuffled[11][0][1:]

In [ ]:
current_dir = os.getcwd()
folder_path = os.path.join(current_dir, f"questionnaires_shuffle_order")
    
if not os.path.exists(folder_path):
        os.mkdir(folder_path)

for idx, questionnaire in enumerate(questionnaires_shuffled):
    file1 = open(os.path.join(folder_path, f"questionnaire_{idx+1}.txt"), "w") # write mode
    for qu_idx, question in enumerate(questionnaire):
        label = img_idx_to_label(question[0], labels, "data/imagenetv2-matched-frequency-format-val")
        file1.write(f"[{qu_idx + 1}] {(label, ) + questionnaires_shuffled[idx][qu_idx][1:]} \n")
    file1.close()